In [1]:
from IPython.display import display
import ipyleaflet as L
from ipywidgets import widgets as w
import numpy as np

from datacube import Datacube

from odc.ui import show_datasets, mk_image_overlay, with_ui_cbk

In [2]:
#Sentinel-1 datacube
dc = Datacube(config='radar.conf')

#query for the relevant product
query = {'lat' : (-44,-9),
         'long' : (112,154),
         'time': ('2016-09-01','2019-07-01'),
         'product' : 's1_gamma0_scene_v2',
         'group_by' : 'solar_day',
         'output_crs' : 'EPSG:3577',
         'resolution' : (25,25)
        }

dss=dc.find_datasets(**query)



In [22]:
gj = show_datasets(dss,
                  style={'opacity': 0.3, 'fillOpacity': 0},
                  width='600px', 
                  height='600px', 
                  scroll_wheel_zoom=True,
                  mode='geojson')

In [23]:
from datacube.utils import geometry
from datacube.testutils.geom import epsg4326

In [81]:
geoms = [geometry.Geometry(poly,epsg4326) for poly in gj.data]
overall_coverage_geom = geometry.unary_union(geoms)

#map the overall coverage from this query
m = L.Map(center=(-30,133),zoom=3)
layer_data = overall_coverage_geom.__geo_interface__
gj_layer = L.GeoJSON(data=layer_data)
m.add_layer(gj_layer)


In [62]:
display(m)

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [92]:
import fiona

#open the shapefile with all the Ramsar wetlands
shp_path = 'ramsar_wetlands.shp'
shapes = fiona.open(shp_path,'r')

crs = geometry.CRS(shapes.crs_wkt)

geoms = [geometry.Geometry(shape['geometry'],crs=crs).to_crs(epsg4326) for shape in shapes]
int_geoms = [geometry.unary_intersection([overall_coverage_geom,shape]) for shape in geoms]
#this is a hack because 7 is the type of an empty geometry but isn't mapped to anything in
#Geometry's internal dictionary of what the types mean
empty_list = [g._geom.GetGeometryType()==7 for g in int_geoms]
int_geoms = [shape for i,shape in enumerate(int_geoms) if not empty_list[i]]
int_geom = geometry.unary_union(int_geoms)

In [93]:
#map the overall coverage from this query
m = L.Map(center=(-30,133),zoom=3)
layer_data = int_geom.__geo_interface__
gj_layer = L.GeoJSON(data=layer_data)
m.add_layer(gj_layer)


In [94]:
display(m)

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …